In [1]:
import numpy as np

import cv2

import glob
import sys

In [3]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((6*7,3), np.float32)

objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('./calibration/*.jpg') #파일에 있는 jpg 모두 불러오기

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
        
        
#stdoutOrigin=sys.stdout
#sys.stdout=open("result.txt","w")#왜곡계수들 txt파일에 저장

#카메라 매크릭스와 왜곡계수        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

#print("ret: ",ret)
print("mtx: ",mtx) #카메라 매트릭스
np.savetxt("savemtx.txt",mtx)
np.save("savemtx.npy",mtx)
print("dist: ",dist)
np.savetxt("./calibration/result/savedist.txt",dist)
np.save("./calibration/result/savedist.npy",dist)
#print("rvecs: ",rvecs) #회전벡터
#print("tvecs: ",tvecs) #이동벡터

img = cv2.imread('./calibration/KakaoTalk_20201011_204212702_16.jpg')#보정할 사진
h, w = img.shape[:2]

newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]

cv2.imwrite('calibresult.png',dst) #calibresult.png로 저장됨

tot_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error
print("total error: {0}".format(tot_error/len(objpoints)))

#sys.stdout.close()
#sys.stdout=stdoutOrigin

cv2.destroyAllWindows()

mtx:  [[1.15974662e+03 0.00000000e+00 5.89729853e+02]
 [0.00000000e+00 1.19962596e+03 7.17526917e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:  [[ 0.17063425 -1.50810018  0.02381674  0.02423583  2.95143698]]
total error: 0.0849303638598435


In [59]:
#앞에서 구한 내부 파라미터들을 사용하여 보정
#dist=[]
#mtx: 카메라 행렬 저장-fx,fy,cx,cy
#K=mtx 
K=np.load("./calibration/result/savemtx.npy")
dist=np.load("./calibration/result/savedist.npy")

image=cv2.imread('KakaoTalk_20201011_233316016_13.jpg')
resize=cv2.resize(image,(640,360))
images=glob.glob('KakaoTalk_20201011_233316016_13.jpg')

for fname in images:
    img=cv2.imread(fname)
    h,w=img.shape[:2]
    
        # undistort
    newcamera, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w,h), 0)
    newimg = cv2.undistort(img, K, dist, None, newcamera)
    resize2 = cv2.resize(newimg, (640, 360))
    
    # save image
    newfname = fname+'.undis.png'
    cv2.imwrite(newfname, newimg)

#cv2.imshow("prior_img",resize) #결과 보여주기
#cv2.imshow("new_img",resize2)
cv2.waitKey()

-1

In [ ]:
cv2.destroyAllWindows()